# Singular locus of plane curves over finite fields

Our main goal is calculate the singular locus of plane curves over finite fields.

#### Caution
This ipynb project needs SageMath (at least 9.1) kernel to run.

#### Author
Doyo Yutaro  
Yokohama National University Graduate School of Environment and Information Science

<!--- below are mathjax newcommands --->
$\newcommand{\ff}{\mathbb{F}}$
$\newcommand{\sing}{\mathrm{Sing}\,}$

## singular_locus()
 
return the singular locus of a curve over the algebraically closed field of the domain of argument curve  
 
**Input:** a polynomial $f \in k[x,y]$ (where $k = \ff_q$ is a finite field)  
**Output:** the singular locus $\sing V(f)$ and the extension field of $k$ in which all points of the singular locus are rational.

In [4]:
def singular_locus(f):
    if not f.nvariables() == 2:
        raise ValueError("argument must be a polynomial of two variable")
    
    L = f.base_ring()
    x, y = f.variables()
    q = L.order()
    
    I = ideal(f, diff(f, x), diff(f, y))
    G = I.groebner_basis()
    g = [g for g in G if g.nvariables() == 1][0]
    H = list(factor(g))
    mod = []

    Y, i, exdeg = [], 0, 1
    for h in H:
        h_ = h[0].change_ring(L) # extend base field
        y = h_.variables()[0]
        deg = h_.degree()
        if deg == 1:
            Y.append(- h_.subs({y: 0}))
        else:
            if not gcd(exdeg, h_.degree()) == 1:
                h_ = list(h_.factor())[0]
            h_ = h_.univariate_polynomial()
            L = L.extension(h_, 'a_' + str(i))
            mod.append(L.modulus())
            if isinstance(L.gen(), tuple):
                a = L.gen()[0]
            else:
                a = L.gen()
            i += 1
            Y.extend(a**q**j for j in range(deg)) #ここ違うよ！
            q = L.order()
            exdeg *= deg
            
    sing = []
    G = [g.change_ring(L) for g in G[:-1]]
    x,y = G[0].variables()

    for p in Y:
        g = ideal(g_.subs({y: p}) for g_ in G).groebner_basis()[0].change_ring(L).univariate_polynomial()
        if g.degree() == 1:
            sing.append((- g.subs({x: 0}), p))
            continue
        else:
            x = g.variables()[0]
            H = list(factor(g))
            for h in H:
                h = h[0]
                x = h.variables()[0]
                deg = h.degree()
                if deg == 1:
                    sing.append((- h.subs({x: 0}), p))
                else:
                    if not gcd(exdeg, h.degree()) == 1:
                        h = list(h.factor())[0][0]
                    L = L.extension(h, 'a_' + str(i))
                    mod.append(L.modulus())
                    if isinstance(L.gen(), tuple):
                        a = L.gen()[0]
                    else:
                        a = L.gen()
                    i += 1
                    sing.extend((a**q**i, p) for i in range(deg))
                    q = L.order()
                    exdeg *= deg
    
    return sing, mod

## test cell

Let $K = \ff_5$, $R = K[x, y]$ and set lex order where $x > y$. Now calculate the singular locus of $f = (x^3 - y^2)(x - 2)$.

In [6]:
K = GF(5)
R.<x, y> = PolynomialRing(K, 2, order="lex")
f = (x^3 - y^2)*(x - 2)
singular_locus(f)

([(0, 0), (2, a_0), (2, 4*a_0)], [x^2 + 2])

Next we consider $f = x^4y^2 + 2x^4 + 2y^5 + 2y$ which needs to extend the base field twice in a calculation.  

In [7]:
K = GF(5)
R.<x, y> = PolynomialRing(K, 2, order="lex")
f = x^4*y^2 + 2*x^4 + 2*y^5 + 2*y
singular_locus(f)

([(a_1, a_0),
  (3*a_1, a_0),
  (4*a_1, a_0),
  (2*a_1, a_0),
  (4*a_0*a_1, 4*a_0),
  (3*a_0*a_1, 4*a_0),
  (2*a_0*a_1, 4*a_0),
  (a_0*a_1, 4*a_0)],
 [x^2 + 2, a_1^4 + 2*a_0])

## check cell

In [ ]:
'''
    check procedure
'''

K = GF(5)
R.<x, y> = PolynomialRing(K, 2, order="lex")
f = (x^3 - y^2)*(x - 2)
I = ideal(f, diff(f, x), diff(f, y))
G = I.groebner_basis()
g = G[-1]

H = list(factor(g))
h = H[0][0]
Y = []

L = K
h = H[0][0].change_ring(L)
y = h.variables()
# H[0][0] = y
# H[0][0].degree == 1


# Y.append(- H[0][0].subs({y: 0})) 

# # H[1][0] = y^2 + 2
# # H[1][0].degree == 2
# S.<z> = K[]
# L = K.extension(H[1][0].subs({y: z}), 'a')
# a = L.gen()
# Y.extend(a**5**i for i in range(2))

# # Y = [0, a, 4*a]

# sing = []
# G_ = [g.change_ring(L) for g in G[:-1]]
# x,y = G_[0].variables()

